<center><h2> **Transfer Learning** </h2></center>
<h3> Note </h3>
<ul> 
    <li>Resnet18 pretrained with Imagenet </li>
    <li> Images resized to 224, with resnet's normalization. Resize doesn't work with multiprocessing, so data loading couldn't be parallelized. </li>
  </ul>
<h3> Configurations</h3>
1. Only last custom linear layer trained : 77.75 %
2. All layers trained with Adam : ~ 95% 

In [10]:
!pip3 install torch==1.3.1+cu92 torchvision==0.4.2+cu92 -f https://download.pytorch.org/whl/torch_stable.html --user

Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Using cached https://download.pytorch.org/whl/cu92/torch-1.3.1%2Bcu92-cp36-cp36m-win_amd64.whl
  Using cached https://download.pytorch.org/whl/cu92/torchvision-0.4.2%2Bcu92-cp36-cp36m-win_amd64.whl
  Found existing installation: torch 1.3.1
    Uninstalling torch-1.3.1:
      Successfully uninstalled torch-1.3.1


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'c:\\users\\zhlow2\\appdata\\roaming\\python\\python36\\site-packages\\~orch\\lib\\c10.dll'
Check the permissions.

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [1]:
%matplotlib inline
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

In [2]:
!tar -zxvf ./cifar10-python/cifar-10-python.tar.gz

x cifar-10-batches-py/
x cifar-10-batches-py/data_batch_4
x cifar-10-batches-py/readme.html
x cifar-10-batches-py/test_batch
x cifar-10-batches-py/data_batch_3
x cifar-10-batches-py/batches.meta
x cifar-10-batches-py/data_batch_2
x cifar-10-batches-py/data_batch_5
x cifar-10-batches-py/data_batch_1


In [3]:

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root='.', train=True, download=False, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='.', train=False, download=False, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)


In [4]:
torch.cuda.is_available()

True

In [5]:
!mkdir checkpoint

A subdirectory or file checkpoint already exists.


In [11]:
os.get_terminal_size().columns

282

In [18]:
'''EfficientNet in PyTorch.
Paper: "EfficientNet: Rethinking Model Scaling for Convolutional Neural Networks".
'''
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import sys
import time
import math

import torch.nn as nn
import torch.nn.init as init
device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
term_width =os.get_terminal_size().columns
term_width = int(term_width)


TOTAL_BAR_LENGTH = 65.
last_time = time.time()
begin_time = last_time
def main(resume,lr,epochAdd):
    global best_acc, start_epoch, term_width, TOTAL_BAR_LENGTH,last_time ,begin_time,device
    


    def get_mean_and_std(dataset):
        '''Compute the mean and std value of dataset.'''
        dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=True, num_workers=2)
        mean = torch.zeros(3)
        std = torch.zeros(3)
        print('==> Computing mean and std..')
        for inputs, targets in dataloader:
            for i in range(3):
                mean[i] += inputs[:,i,:,:].mean()
                std[i] += inputs[:,i,:,:].std()
        mean.div_(len(dataset))
        std.div_(len(dataset))
        return mean, std

    def init_params(net):
        '''Init layer parameters.'''
        for m in net.modules():
            if isinstance(m, nn.Conv2d):
                init.kaiming_normal(m.weight, mode='fan_out')
                if m.bias:
                    init.constant(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                init.constant(m.weight, 1)
                init.constant(m.bias, 0)
            elif isinstance(m, nn.Linear):
                init.normal(m.weight, std=1e-3)
                if m.bias:
                    init.constant(m.bias, 0)

    def progress_bar(current, total, msg=None):
        global last_time, begin_time
        if current == 0:
            begin_time = time.time()  # Reset for new bar.

        cur_len = int(TOTAL_BAR_LENGTH*current/total)
        rest_len = int(TOTAL_BAR_LENGTH - cur_len) - 1

        sys.stdout.write(' [')
        for i in range(cur_len):
            sys.stdout.write('=')
        sys.stdout.write('>')
        for i in range(rest_len):
            sys.stdout.write('.')
        sys.stdout.write(']')

        cur_time = time.time()
        step_time = cur_time - last_time
        last_time = cur_time
        tot_time = cur_time - begin_time

        L = []
        L.append('  Step: %s' % format_time(step_time))
        L.append(' | Tot: %s' % format_time(tot_time))
        if msg:
            L.append(' | ' + msg)

        msg = ''.join(L)
        sys.stdout.write(msg)
        for i in range(term_width-int(TOTAL_BAR_LENGTH)-len(msg)-3):
            sys.stdout.write(' ')

        # Go back to the center of the bar.
        for i in range(term_width-int(TOTAL_BAR_LENGTH/2)+2):
            sys.stdout.write('\b')
        sys.stdout.write(' %d/%d ' % (current+1, total))

        if current < total-1:
            sys.stdout.write('\r')
        else:
            sys.stdout.write('\n')
        sys.stdout.flush()

    def format_time(seconds):
        days = int(seconds / 3600/24)
        seconds = seconds - days*3600*24
        hours = int(seconds / 3600)
        seconds = seconds - hours*3600
        minutes = int(seconds / 60)
        seconds = seconds - minutes*60
        secondsf = int(seconds)
        seconds = seconds - secondsf
        millis = int(seconds*1000)

        f = ''
        i = 1
        if days > 0:
            f += str(days) + 'D'
            i += 1
        if hours > 0 and i <= 2:
            f += str(hours) + 'h'
            i += 1
        if minutes > 0 and i <= 2:
            f += str(minutes) + 'm'
            i += 1
        if secondsf > 0 and i <= 2:
            f += str(secondsf) + 's'
            i += 1
        if millis > 0 and i <= 2:
            f += str(millis) + 'ms'
            i += 1
        if f == '':
            f = '0ms'
        return f
    
    class Block(nn.Module):
        '''expand + depthwise + pointwise + squeeze-excitation'''

        def __init__(self, in_planes, out_planes, expansion, stride):
            super(Block, self).__init__()
            self.stride = stride

            planes = expansion * in_planes
            self.conv1 = nn.Conv2d(
                in_planes, planes, kernel_size=1, stride=1, padding=0, bias=False)
            self.bn1 = nn.BatchNorm2d(planes)
            self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                                   stride=stride, padding=1, groups=planes, bias=False)
            self.bn2 = nn.BatchNorm2d(planes)
            self.conv3 = nn.Conv2d(
                planes, out_planes, kernel_size=1, stride=1, padding=0, bias=False)
            self.bn3 = nn.BatchNorm2d(out_planes)

            self.shortcut = nn.Sequential()
            if stride == 1 and in_planes != out_planes:
                self.shortcut = nn.Sequential(
                    nn.Conv2d(in_planes, out_planes, kernel_size=1,
                              stride=1, padding=0, bias=False),
                    nn.BatchNorm2d(out_planes),
                )

            # SE layers
            self.fc1 = nn.Conv2d(out_planes, out_planes//16, kernel_size=1)
            self.fc2 = nn.Conv2d(out_planes//16, out_planes, kernel_size=1)

        def forward(self, x):
            out = F.relu(self.bn1(self.conv1(x)))
            out = F.relu(self.bn2(self.conv2(out)))
            out = self.bn3(self.conv3(out))
            shortcut = self.shortcut(x) if self.stride == 1 else out
            # Squeeze-Excitation
            w = F.avg_pool2d(out, out.size(2))
            w = F.relu(self.fc1(w))
            w = self.fc2(w).sigmoid()
            out = out * w + shortcut
            return out


    class EfficientNet(nn.Module):
        def __init__(self, cfg, num_classes=10):
            super(EfficientNet, self).__init__()
            self.cfg = cfg
            self.conv1 = nn.Conv2d(3, 32, kernel_size=3,
                                   stride=1, padding=1, bias=False)
            self.bn1 = nn.BatchNorm2d(32)
            self.layers = self._make_layers(in_planes=32)
            self.linear = nn.Linear(cfg[-1][1], num_classes)

        def _make_layers(self, in_planes):
            layers = []
            for expansion, out_planes, num_blocks, stride in self.cfg:
                strides = [stride] + [1]*(num_blocks-1)
                for stride in strides:
                    layers.append(Block(in_planes, out_planes, expansion, stride))
                    in_planes = out_planes
            return nn.Sequential(*layers)

        def forward(self, x):
            out = F.relu(self.bn1(self.conv1(x)))
            out = self.layers(out)
            out = out.view(out.size(0), -1)
            out = self.linear(out)
            return out


    def EfficientNetB0():
        # (expansion, out_planes, num_blocks, stride)
        cfg = [(1,  16, 1, 2),
               (6,  24, 2, 1),
               (6,  40, 2, 2),
               (6,  80, 3, 2),
               (6, 112, 3, 1),
               (6, 192, 4, 2),
               (6, 320, 1, 2)]
        return EfficientNet(cfg)
    net = EfficientNetB0()
    net = net.to(device)
    if device == 'cuda':
        net = torch.nn.DataParallel(net)
        cudnn.benchmark = True

    if resume:
        # Load checkpoint.
        print('==> Resuming from checkpoint..')
        assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
        checkpoint = torch.load('./checkpoint/ckpt.pth')
        net.load_state_dict(checkpoint['net'])
        best_acc = checkpoint['acc']
        start_epoch = checkpoint['epoch']
    else:
        best_acc = 0  # best test accuracy
        start_epoch = 0  # start from epoch 0 or last checkpoint epoch

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)

    # Training
    def train(epoch):
        print('\nEpoch: %d' % epoch)
        net.train()
        train_loss = 0
        correct = 0
        total = 0
        for batch_idx, (inputs, targets) in enumerate(trainloader):
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

#             progress_bar(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
#                 % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))

    def test(epoch):
        global best_acc
        net.eval()
        test_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for batch_idx, (inputs, targets) in enumerate(testloader):
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = net(inputs)
                loss = criterion(outputs, targets)

                test_loss += loss.item()
                _, predicted = outputs.max(1)
                total += targets.size(0)
                correct += predicted.eq(targets).sum().item()

#                 progress_bar(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
#                     % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

        # Save checkpoint.
        acc = 100.*correct/total
        print('acc: '+str(acc) + ' last best: '+ str(best_acc))
        if acc > best_acc:
            print('Saving..')
            state = {
                'net': net.state_dict(),
                'acc': acc,
                'epoch': epoch,
            }
            if not os.path.isdir('checkpoint'):
                os.mkdir('checkpoint')
            torch.save(state, './checkpoint/ckpt.pth')
            best_acc = acc


    for epoch in range(start_epoch, start_epoch+epochAdd):
        print('epoch: '+str(epoch))
        train(epoch)
        test(epoch)
lrate= [0.001,0.001,0.0005]
epochAdd=[1,50,100]
nettt = []
for i in range(3):
    nettt.append(main(True,lrate[i],epochAdd[i]))

==> Resuming from checkpoint..
epoch: 214

Epoch: 214
acc: 86.54 best: 86.87
==> Resuming from checkpoint..
epoch: 214

Epoch: 214
acc: 86.54 best: 86.87
epoch: 215

Epoch: 215
acc: 86.62 best: 86.87
epoch: 216

Epoch: 216
acc: 86.58 best: 86.87
epoch: 217

Epoch: 217
acc: 86.46 best: 86.87
epoch: 218

Epoch: 218
acc: 86.91 best: 86.87
Saving..
epoch: 219

Epoch: 219
acc: 86.54 best: 86.91
epoch: 220

Epoch: 220
acc: 86.5 best: 86.91
epoch: 221

Epoch: 221
acc: 86.54 best: 86.91
epoch: 222

Epoch: 222
acc: 86.73 best: 86.91
epoch: 223

Epoch: 223
acc: 86.88 best: 86.91
epoch: 224

Epoch: 224
acc: 86.54 best: 86.91
epoch: 225

Epoch: 225
acc: 86.88 best: 86.91
epoch: 226

Epoch: 226
acc: 86.53 best: 86.91
epoch: 227

Epoch: 227
acc: 86.67 best: 86.91
epoch: 228

Epoch: 228
acc: 86.62 best: 86.91
epoch: 229

Epoch: 229
acc: 86.89 best: 86.91
epoch: 230

Epoch: 230
acc: 86.87 best: 86.91
epoch: 231

Epoch: 231
acc: 86.66 best: 86.91
epoch: 232

Epoch: 232
acc: 86.86 best: 86.91
epoch: 233

In [ ]:
!ls ../input/temptraining

In [ ]:
# class Model(nn.Module):
#     def __init__(self):
#         super().__init__()
#         base = models.resnet18(pretrained=True)
#         self.base = nn.Sequential(*list(base.children())[:-1])
#         in_features = base.fc.in_features
#         self.drop = nn.Dropout()
#         self.final = nn.Linear(in_features,10)
    
#     def forward(self,x):
#         x = self.base(x)
#         x = self.drop(x.view(-1,self.final.in_features))
#         return self.final(x)
    
model = EfficientNetB0().cuda()
[x for x,y in model.named_children()]

In [ ]:
criterion = nn.CrossEntropyLoss()
param_groups = [
    {'params':model.base.parameters(),'lr':.0001},
    {'params':model.final.parameters(),'lr':.001}
]
optimizer = Adam(param_groups)
lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)
states = {}

<h2> Training the model</h2>

In [ ]:
%%time
best_val_acc = -1000
best_val_model = None
for epoch in range(10):  
    model.train(True)
    running_loss = 0.0
    running_acc = 0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.cuda(),labels.cuda()

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item() * inputs.size(0)
        out = torch.argmax(outputs.detach(),dim=1)
        assert out.shape==labels.shape
        running_acc += (labels==out).sum().item()
    print(f"Train loss {epoch+1}: {running_loss/len(trainset)},Train Acc:{running_acc*100/len(trainset)}%")
    
    correct = 0
    model.train(False)
    with torch.no_grad():
        for inputs,labels in valloader:
            out = model(inputs.cuda()).cpu()
            out = torch.argmax(out,dim=1)
            acc = (out==labels).sum().item()
            correct += acc
    print(f"Val accuracy:{correct*100/len(valset)}%")
    if correct>best_val_acc:
        best_val_acc = correct
        best_val_model = deepcopy(model.state_dict())
    lr_scheduler.step()
    
print('Finished Training')  

<h2> Testing the model</h2>

In [ ]:
%%time
correct = 0
model.load_state_dict(best_val_model)
model.train(False)
with torch.no_grad():
    for inputs,labels in testloader:
        out = model(inputs.cuda()).cpu()
        out = torch.argmax(out,dim=1)
        acc = (out==labels).sum().item()
        
        correct += acc
print(f"Test accuracy: {correct*100/len(testset)}%")

In [ ]:
torch.save(best_val_model,'./wtfIsWrongWithKaggleNotebook') 

In [ ]:
transform_test = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
pp = np.load('/kaggle/input/test-data/y_test.npy')
pp_torch = transform_test(pp[0])
# nettt[len(nettt)-1].eval()
# with torch.no_grad():
#     input = pp_torch.to(device)
#     outputs = nettt[len(nettt)-1](inputs)
#     _, predicted = outputs.max(1)


model.load_state_dict(best_val_model)
model.train(False)
with torch.no_grad():
#     for inputs,labels in testloader:
    out = model(pp_torch.cuda()).cpu()
    out = torch.argmax(out,dim=1)
#     acc = (out==labels).sum().item()




In [ ]:
!ls ../input/test-data

In [ ]:
predicted